Import the necessary libraries, keeping the python standard ones above the non-standard.

In [1]:
from glob import glob
import os
import subprocess
from datetime import datetime

import pandas as pd
import numpy as np

Assign path variables. This way we can easily change the path later if needed.

In [2]:
REPORT_PATH = "//filesrv/MercuryGate/"
PARQUET_PATH = "//filesrv/MercuryGate/Separated/Backup.parquet"
ORDERS_PATH = "C:/OneDrive - Metropolitan Warehouse/Vendor Control/Data Files/POM Level/Pull Sheet Data LH Team/"
TICKETS_PATH = "C:/OneDrive - Metropolitan Warehouse/Vendor Control/Data Files/POM Level/Closing Tickets/"
OUTPUT_PATH = "C:/OneDrive - Metropolitan Warehouse/Vendor Control/Data Files/POM Level/TMSProduct/"
AUTO_PULL_PATH = "C:/OneDrive - Metropolitan Warehouse/Vendor Control/Data Files/POM Level/pompy/get_missing_pull_sheets.py"
AUTO_TICKETS_PATH = "C:/OneDrive - Metropolitan Warehouse/Vendor Control/Data Files/POM Level/pompy/get_missing_closing_tickets.py"
AUTO_DIST_PATH = "C:/OneDrive - Metropolitan Warehouse/Vendor Control/Data Files/POM Level/pompy/get_missing_dist.py"

Read all reports in the report path and create a dataframe.

In [3]:
report_files = glob(os.path.join(REPORT_PATH, "*.csv"))
if len(report_files) > 0:
    df = pd.concat((pd.read_csv(f) for f in report_files), ignore_index=True)
    df = df.drop_duplicates()
else:
    print("Nothing to read. Exiting...")
    exit(0)

Concat the old data to the newer one, replacing any data in the old with the new one.<br>
Archive it in a parquet file.

In [4]:
if os.path.exists(PARQUET_PATH):
        pqt_df = pd.read_parquet(PARQUET_PATH)
        pqt_df = pqt_df.loc[~pqt_df['Primary Reference'].isin(df['Primary Reference'])]
        pqt_df = pd.concat([pqt_df, df], ignore_index=True)
        pqt_df.astype(str).to_parquet(PARQUET_PATH, index=False)
else:
        df.astype(str).to_parquet(PARQUET_PATH, index=False)

Get the columns in the cols list only.

In [5]:
cols = ['Primary Reference', 'Pinnacle Manifest No', 'Target Ship (Late)',
        'Arrival Date', 'Target Delivery (Late)', 'Actual Delivery', 'Origin Code',
        'Origin Zip', 'Dest Code', 'Dest Zip', 'Carrier', 'Carrier Total',
        'Invoice Charge', 'Invoice Date', 'Invoice Number', 'Invoice Total Line Haul',
        'Invoice Total Fuel']
df = df[cols]

Before we can use the Carrier Total, we need to replace it with the Invoice Charge.<br>
Before we can use Invoice Charge, we need to aggregate it over the load.<br>
Before we can aggregate it over the load, we need to divide it over multiple loads if its loads > 1.<br>

In [6]:
#divide the invoices equally over their loads, no effect if only 1 load
number_of_loads_against_invoice = df.groupby('Invoice Number')['Primary Reference'].transform('nunique')
df['Invoice Charge'] = df['Invoice Charge'] / number_of_loads_against_invoice
df['Invoice Total Line Haul'] = df['Invoice Total Line Haul'] / number_of_loads_against_invoice
df['Invoice Total Fuel'] = df['Invoice Total Fuel'] / number_of_loads_against_invoice

In [7]:
#aggregate costs against reference, summing against unique invoices
df_gb = df.groupby([
    'Primary Reference',
    'Invoice Number'])[[
        'Invoice Charge',
        'Invoice Total Line Haul',
        'Invoice Total Fuel']].max().groupby(level='Primary Reference').sum()
df = df.drop(columns=['Invoice Charge',
                      'Invoice Total Line Haul',
                      'Invoice Total Fuel'])
df = df.merge(df_gb, on='Primary Reference')

In [8]:
#aggregate invoice number to create a list, and aggregate invoice date to get max
df['Invoice Number'] = df.groupby('Primary Reference')['Invoice Number'].transform(lambda x: ";".join(set(x)))
df['Invoice Date'] = df.groupby('Primary Reference')['Invoice Date'].transform('max')

In [9]:
#replace 0s with nans to fillna later
df['Invoice Charge'] = df['Invoice Charge'].replace(to_replace=0, value=np.nan)

In [10]:
#replace carrier total with invoice charge if invoice charge is not na
df['Carrier Total'] = df['Invoice Charge'].fillna(df['Carrier Total'])

Drop duplicates from the dataframe using the subset of columns and keeping the last ones.

In [11]:
df.drop_duplicates(subset=['Primary Reference', 'Pinnacle Manifest No', 'Origin Code',
                           'Dest Code','Target Ship (Late)'], inplace=True, keep='last')

Fill any missing codes with their respective zips.

In [12]:
df['Origin Code'].fillna(df['Origin Zip'], inplace=True)
df['Dest Code'].fillna(df['Dest Zip'], inplace=True)

Join Origin Code and Dest Code using " | " to create from_to.<br>
Join Origin Zip and Dest Zip using ";" to create from_to_zip.<br>
Join origin code with origin zip and dest code with dest zip using ";".<br>

In [13]:
df["from_to"] = df["Origin Code"].astype('str') + " | " + df["Dest Code"].astype('str')
df["from_to_zip"] = df["Origin Zip"].astype('str') + ";" + df["Dest Zip"].astype('str')
df["Code;Zip"] = df["Dest Code"].astype('str') + ";" + df["Dest Zip"].astype('str')
df["OriginCode;Zip"] = df["Origin Code"].astype('str') + ";" + df["Origin Zip"].astype('str')

Convert datetime columns to datetime.

In [14]:
df["Target Ship (Late)"] = pd.to_datetime(df["Target Ship (Late)"])
df["Target Delivery (Late)"] = pd.to_datetime(df["Target Delivery (Late)"])
df["Invoice Date"] = pd.to_datetime(df["Invoice Date"])

### create_lane(load):

This function uses the Primary Reference (the load ID) to find the ship datetimes, delivery datetimes,<br>
origin codes + zips, and dest codes + zips against the reference.<br>
Then, it appends the delivery dates to the ship dates and destinations to origins.<br>
Then, it sorts them by the dates in an ascending order.<br>
Then, it loops over the codes, adding them to a list if the current code is not the same as the previous code.<br>
It also splits the code using ";" to get the code and the zip parts separated.<br>
Finally, it joins the codes list using " | " and the zips list using ";".<br>
It then returns the two strings.<br>

In [15]:
def create_lane(load):
    rows = df.loc[df['Primary Reference']==load, ["Target Ship (Late)", "Target Delivery (Late)",
                                                    "OriginCode;Zip", "Code;Zip"]]
    targets = pd.concat([rows["Target Ship (Late)"], rows["Target Delivery (Late)"]])
    code_zips = pd.concat([rows["OriginCode;Zip"], rows["Code;Zip"]])
    rows = pd.DataFrame({"Target": targets, "Code": code_zips}).sort_values(by="Target")
    l = ['']; li = ['']
    for x in rows["Code"]:
        if l[-1] != x.split(";")[0]:
            l.append(x.split(";")[0])
            li.append(x.split(";")[1])
    l = l[1:]
    li = li[1:]
    lane = " | ".join(l)
    zips = ";".join(li)
    return lane, zips

df['lane'], df['zips'] = zip(*df['Primary Reference'].apply(create_lane))

### process_manifest(manifest):

This function takes the manifest, converts it to str, and splits it by ", ".<br>
Then, it loops over the list created to process each manifest.<br>

> If "VEN" is found in the capitalized manifest, "Yes" is appended to the is_vendor_pickup list,<br>
> the manifest is converted to 0, and '- 53FT' is appended to the size list. Otherwise, "No" is appended.<br>
> If 'ft' is found in the lower-cased manifest, 7 digits of the manifest are taken, and the whole<br>
> manifest is taken into the size list. Otherwise, 6 digits are taken, and '- 53FT' is appended.<br>

Finally, it returns the manifest, size, and is_vendor_pickup lists.<br>
<br>
__Note that the sizes of these lists should be the same to avoid problems later when exploding them.__

In [16]:
def process_manifest(manifest):
    manifest = str(manifest).strip().split(", ")
    size = []
    is_vendor_pickup = []
    for i, j in enumerate(manifest):
        if "VEN" in j.upper() or "PU" in j.upper():
            manifest[i] = 0
            is_vendor_pickup.append("Yes")
            size.append('- 53FT')
        else:
            is_vendor_pickup.append("No")
            if 'ft' in j.lower():
                manifest[i] = ''.join(filter(str.isdigit, j))[:7]
                size.append(j)
            else:
                manifest[i] = ''.join(filter(str.isdigit, j))[:6]
                size.append('- 53FT')
    return manifest, size, is_vendor_pickup

df['Pinnacle Manifest No'], df['size'], df['Vendor Pickup?'] = zip(*df['Pinnacle Manifest No'].apply(process_manifest))


Explode the three lists generated. This means that the lists are opened up and the elements fill rows.

In [17]:
df = df.explode(['Pinnacle Manifest No', 'size', 'Vendor Pickup?'])

Convert the manifest numbers to numeric, coercing any errors.<br>
Coercing errors means that they're converted to NaN. We fillna with 0,<br>
and convert the numbers to integers. Finally, we rename the column to 'manifest_num'.<br>

In [18]:
df['Pinnacle Manifest No'] = pd.to_numeric(df['Pinnacle Manifest No'], errors='coerce'
                                           ).fillna(0).astype(int)
df = df.rename(columns={'Pinnacle Manifest No': 'manifest_num'})

Show the first 10 rows of the df.

In [19]:
print(df.head(10))

    Primary Reference  manifest_num  Target Ship (Late)      Arrival Date  \
1   LD18189 (Load ID)             0 2022-10-20 15:00:00  2022-10-21 06:00   
3   LD18196 (Load ID)             0 2022-10-21 11:00:00  2022-10-26 08:00   
5   LD18204 (Load ID)             0 2022-10-21 15:00:00  2022-10-25 08:00   
7   LD18205 (Load ID)             0 2022-10-21 13:00:00  2022-10-24 08:00   
9   LD18226 (Load ID)             0 2022-10-23 08:00:00  2022-10-24 07:30   
11  LD18248 (Load ID)             0 2022-10-22 23:45:00  2022-10-25 08:00   
13  LD18271 (Load ID)             0 2022-10-24 11:00:00  2022-10-27 08:00   
15  LD18273 (Load ID)             0 2022-10-24 11:00:00  2022-10-27 09:00   
17  LD18288 (Load ID)             0 2022-10-24 16:00:00  2022-10-25 07:00   
19  LD18294 (Load ID)             0 2022-10-21 15:00:00  2022-10-22 06:00   

   Target Delivery (Late)   Actual Delivery Origin Code  Origin Zip Dest Code  \
1     2022-10-21 08:00:00  2022-10-21 08:00       ED-SC       29492    

Print the shape of the dataframe (rows and columns).

In [20]:
print(df.shape)

(8850, 25)


Create a temporary df with refined manifest_nums, and store it in a csv so that it<br>
can be accessed by the auto pulling scripts.

In [21]:
tempdf = pd.DataFrame()
tempdf["manifest_num"] = df["manifest_num"].drop_duplicates().dropna()
tempdf.to_csv(OUTPUT_PATH + "temp.csv", index=False)

Run the auto pulling scripts to get the Pull Sheet Data and the Closing Tickets.

In [22]:
subprocess.run(["python", AUTO_PULL_PATH, OUTPUT_PATH], stdout=subprocess.PIPE)
subprocess.run(["python", AUTO_TICKETS_PATH], stdout=subprocess.PIPE)

CompletedProcess(args=['python', 'C:/OneDrive - Metropolitan Warehouse/Vendor Control/Data Files/POM Level/pompy/get_missing_closing_tickets.py'], returncode=0, stdout=b'\r\nGetting the POMs...\r\n\r\nGetting Closing Tickets...\r\n(0,)\r\nSeries([], Name: OrderNo, dtype: object)\r\n\r\nNo missing nums found. Exiting...\r\n\r\n')

Remove the temporary csv.

In [23]:
os.remove(OUTPUT_PATH + "temp.csv")

### get_leg(list_stops, delim):

([A, B, C, A], " | ") => [A | B, B | C, C | A]<br>
Takes a list of stops and a delimiter to join the stops together using the delimiter.<br>

In [24]:
def get_leg(list_stops, delim):
    list_legs = []
    for i in range(len(list_stops)-1):
        list_legs.append(list_stops[i] + delim + list_stops[i+1])
    return list_legs

### revise_from_to(lane, zip_lane, from_to, from_to_zip, load):

Checks if the from_to is not in the lane and processes it, otherwise<br>
returns [from_to], [from_to_zip], and [].<br>
<br>
Finds the index of the origin and the destination+1 in the lane's stops.<br>
Then, it creates a list of stops from the origin to destination.<br>
Then, it uses the get_leg function to convert the list of stops to list of legs.<br>
Likwise for the zip_lane.<br>
In case of a ValueError, appends the primary reference to the err list.<br>
<br>
Returns revised_from_to, revised_zips, and erronous Primary Reference.<br>
<br>
__Note that the length of the two lists, revised and revised_zips, must be the same for explosion.__

In [25]:
def revise_from_to(lane, zip_lane, from_to, from_to_zip, load):
    err = []
    revised = [from_to]
    revised_zips = [from_to_zip]
    if from_to not in lane:
        origin, destination = from_to.split(" | ")
        stops = lane.split(" | ")
        zips = zip_lane.split(";")
        try:
            origin_index = stops.index(origin)
            destination_index = origin_index + stops[origin_index:].index(destination) + 1
            revised_stops = stops[origin_index:destination_index]
            revised_zips_list = zips[origin_index:destination_index]
            revised = get_leg(revised_stops, " | ")
            revised_zips = get_leg(revised_zips_list, ";")
        except ValueError:
            err.append(load)
    return revised, revised_zips, err

err_df = pd.DataFrame()
df['revised_from_to'], df['revised_zips'], err_df['Load'] = zip(*df.apply(lambda x: revise_from_to(
    x['lane'], x['zips'], x['from_to'], x['from_to_zip'], x['Primary Reference']), axis=1))

We explode the revised_from_to and revised_zips' list to access the contents inside.

In [26]:
df = df.explode(['revised_from_to', 'revised_zips'])

In [27]:
df['count_leg'] = df.groupby(["Primary Reference", "revised_from_to"]).transform('size')

In [28]:
df.loc[(df['Primary Reference']=="LD27907 (Load ID)"), "count_leg"]

31720    2.0
31720    2.0
Name: count_leg, dtype: float64

In [29]:
distance_table = pd.read_csv("C:/OneDrive - Metropolitan Warehouse/Vendor Control/Data Files/POM Level/Helpers/ZipDistances.csv", dtype={'Zip_Code': str})
print(distance_table.head())

           Zip  Distance
0  91752;85043       323
1  85043;85043         0
2  85043;91752       324
3  85043;27263      2102
4  27263;85043      2119


In [30]:
failed_dist = pd.DataFrame()
failed_dist['dist'] = df.loc[~df['revised_zips'].isin(distance_table['Zip']), 'revised_zips']
failed_dist = failed_dist.drop_duplicates().dropna()

In [31]:
# split column into multiple columns by delimiter
if len(failed_dist.index) > 0:
    failed_dist[['zip1', 'zip2']] = failed_dist['dist'].str.split(';', n=1, expand=True).dropna(how='any')
    failed_dist.drop(columns=['dist'], inplace=True)
    failed_dist.to_csv(OUTPUT_PATH+'temp_dist.csv', index=False)
    subprocess.Popen(["python", AUTO_DIST_PATH]).wait()
    os.remove(OUTPUT_PATH + "temp_dist.csv")

In [32]:
def get_distance(zips):
    try:
        # both Zip and leg should be strings
        dist = distance_table.loc[distance_table['Zip'] == zips, 'Distance'].iloc[0]
        failed_dist = None
    except:
        dist = 0
        failed_dist = zips
    return dist, failed_dist

exfailed_dist = pd.DataFrame()
df["zip_distance"], exfailed_dist['exdist'] = zip(*df['revised_zips'].apply(get_distance))

In [33]:
# calculate adjusted distance for each row
df['Adjusted Distance'] = df['zip_distance'] * df['count_leg']

# drop duplicates
df.drop_duplicates(['Primary Reference', 'revised_from_to'], inplace=True)

# group by Primary Reference and transform the sum of adjusted distance
df['Load Total Distance'] = df.groupby(['Primary Reference'])['Adjusted Distance'].transform('sum')

# reset index
df.reset_index(drop=True, inplace=True)

In [34]:
# calculate number of manifests per leg
legs_per_load = df.groupby(['Primary Reference']).revised_from_to.nunique().reset_index(name='Legs per Load')

# merge manifests_per_leg with fact_table
df = pd.merge(df, legs_per_load, on=['Primary Reference'])

# allocate leg cost to manifests
df['Leg Cost'] = np.where(df['Load Total Distance'] <= 0,
                          df['Carrier Total'] / df['Legs per Load'].clip(lower=1),
                          df['Carrier Total'] * df['Adjusted Distance'] / df['Load Total Distance'])

In [35]:
print("\nGetting the POMs...")
all_files = glob(os.path.join(ORDERS_PATH, "*.csv"))
df_poms = pd.concat((pd.read_csv(f, encoding='ISO-8859-1', engine='c', dtype={'Manifest No': object,
                                                                              'OrderNo': object},
                                usecols=['Manifest No', 'OrderNo', 'Weight', 'Cu_Ft_']
                                ) for f in all_files), ignore_index=True).drop_duplicates().round(2)
df_pomsg = df_poms.groupby(['Manifest No', 'OrderNo'], as_index=False).agg({'Cu_Ft_': 'sum', 'Weight': 'sum'})
df_pomsg['Count'] = df_poms.groupby(['Manifest No', 'OrderNo'], as_index=False).size().loc[:, 'size']
df_poms = df_pomsg.set_index('OrderNo')


Getting the POMs...


In [36]:
print("\nGetting Closing Tickets...")
kpi_files = glob(os.path.join(TICKETS_PATH, "*.csv"))
df_kpi = pd.concat((pd.read_csv(f, encoding='ISO-8859-1', engine='c', dtype={'Order #': object},
                                usecols=['Order #', 'Actual Delivery Date', 'Client Name', 'Amount']
                                ) for f in kpi_files), ignore_index=True).drop_duplicates().round(2)
df_kpi = df_kpi.groupby('Order #').agg({'Actual Delivery Date': 'first', 'Client Name': 'first', 'Amount': 'sum'})


Getting Closing Tickets...


In [37]:
print("\nMerging df_poms and closing_tickets...")
df_poms = df_poms.join(df_kpi)
df_poms.index.names = ['Order #']
df_poms.reset_index(inplace=True)
print(df_poms.shape)


Merging df_poms and closing_tickets...
(1191040, 8)


In [38]:
print("\nMerging fact_table and df_poms...")
df_poms['Manifest No'] = df_poms['Manifest No'].fillna(0).astype(int)
df_poms.set_index('Manifest No', inplace=True)
df.set_index('manifest_num', inplace=True)
fact_table = df.join(df_poms)
fact_table.index.names = ['manifest_num']
fact_table.reset_index(inplace=True)
print(fact_table.shape)


Merging fact_table and df_poms...
(75772, 40)


In [39]:
fact_table['Order Delivery Date'] = (pd.TimedeltaIndex(fact_table['Actual Delivery Date'
                                            ], unit='d') + datetime(1899, 12, 30)).strftime('%Y-%m-%d')
fact_table.drop(columns=['Actual Delivery Date'], inplace=True)

c:\Users\shahmir.tariq\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\arrays\timedeltas.py:908: RuntimeWarning: invalid value encountered in cast
  base = data.astype(np.int64)
c:\Users\shahmir.tariq\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\arrays\timedeltas.py:912: RuntimeWarning: invalid value encountered in cast
  data = (base * m + (frac * m).astype(np.int64)).view("timedelta64[ns]")


In [40]:
fact_table.tail()

,manifest_num,Primary Reference,Target Ship (Late),Arrival Date,Target Delivery (Late),Actual Delivery,Origin Code,Origin Zip,Dest Code,Dest Zip,...,Load Total Distance,Legs per Load,Leg Cost,Order #,Cu_Ft_,Weight,Count,Client Name,Amount,Order Delivery Date
75767,105310,LD28357 (Load ID),2023-04-24 10:00:00,2023-04-28 07:00,2023-04-27 13:00:00,2023-04-28 08:00,CA-EV,91752,NJ-HS,8861,...,2733.0,1,6600.00,3337675,13.50,50.0,1.0,CYMAX STORES,202.82,2023-05-02
75768,105310,LD28357 (Load ID),2023-04-24 10:00:00,2023-04-28 07:00,2023-04-27 13:00:00,2023-04-28 08:00,CA-EV,91752,NJ-HS,8861,...,2733.0,1,6600.00,3337823,12.22,26.0,1.0,CYMAX STORES,178.42,2023-05-03
75769,263282,LD24949 (Load ID),2023-02-10 15:00:00,NaN,2023-02-11 07:00:00,2023-02-11 07:00,GA-SL,31326,NC-SD,27263,...,320.0,1,885.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75770,265679,LD27832 (Load ID),2023-04-06 18:00:00,NaN,2023-04-07 07:00:00,2023-04-06 21:41,GA-SL,31326,NC-SD,27263,...,320.0,1,858.69,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75771,9039767,LD28116 (Load ID),2023-04-14 14:00:00,2023-04-14 14:00,2023-04-15 13:00:00,2023-04-15 07:30,CA-EV,85043,CA-EV,91752,...,0.0,0,2175.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
fact_table['Manifest Cubes'] = fact_table.groupby(['Primary Reference', 'revised_from_to', 'manifest_num'])['Cu_Ft_'].transform('sum')

In [42]:
fact_table['Leg Cubes'] = fact_table.groupby(['Primary Reference', 'revised_from_to'])['Cu_Ft_'].transform('sum')

In [43]:
fact_table.columns

Index(['manifest_num', 'Primary Reference', 'Target Ship (Late)',
       'Arrival Date', 'Target Delivery (Late)', 'Actual Delivery',
       'Origin Code', 'Origin Zip', 'Dest Code', 'Dest Zip', 'Carrier',
       'Carrier Total', 'Invoice Date', 'Invoice Number', 'Invoice Charge',
       'Invoice Total Line Haul', 'Invoice Total Fuel', 'from_to',
       'from_to_zip', 'Code;Zip', 'OriginCode;Zip', 'lane', 'zips', 'size',
       'Vendor Pickup?', 'revised_from_to', 'revised_zips', 'count_leg',
       'zip_distance', 'Adjusted Distance', 'Load Total Distance',
       'Legs per Load', 'Leg Cost', 'Order #', 'Cu_Ft_', 'Weight', 'Count',
       'Client Name', 'Amount', 'Order Delivery Date', 'Manifest Cubes',
       'Leg Cubes'],
      dtype='object')

In [44]:
# calculate number of manifests per leg
manifests_per_leg = fact_table.groupby(['Primary Reference', 'revised_from_to']).manifest_num.nunique().reset_index(name='Manifests Per Leg')

# merge manifests_per_leg with fact_table
fact_table = pd.merge(fact_table, manifests_per_leg, on=['Primary Reference', 'revised_from_to'])

# allocate leg cost to manifests
fact_table['Manifest Cost'] = np.where(fact_table['Leg Cubes'] <= 0,
                                       fact_table['Leg Cost'] / fact_table['Manifests Per Leg'].clip(lower=1),
                                       fact_table['Leg Cost'] * fact_table['Manifest Cubes'] / fact_table['Leg Cubes'])


In [45]:
# calculate number of manifests per leg
orders_per_manifest = fact_table.groupby(['Primary Reference', 'revised_from_to', 'manifest_num'])['Order #'].nunique().reset_index(name='Orders Per Manifest')

# merge orders_per_leg with fact_table
fact_table = pd.merge(fact_table, orders_per_manifest, on=['Primary Reference', 'revised_from_to', 'manifest_num'])

# allocate leg cost to manifests
fact_table['POM Cost'] = np.where(fact_table['Manifest Cubes'] <= 0,
                                  fact_table['Manifest Cost'] / fact_table['Orders Per Manifest'].clip(lower=1),
                                  fact_table['Manifest Cost'] * fact_table['Cu_Ft_'] / fact_table['Manifest Cubes'])

In [46]:
fact_table = fact_table.rename(columns={'revised_from_to': 'Leg', 'revised_zips': 'Leg Zips',
                              'count_leg': 'Leg Instance per Load', 'Cu_Ft_': 'Order Cubes',
                              'zip_distance': 'Actual Distance'})

In [47]:
if os.path.exists(OUTPUT_PATH + "Product.csv"):
    old_data = pd.read_csv(OUTPUT_PATH + "Product.csv")
    references = set(fact_table["Primary Reference"])
    old_data = old_data[~old_data["Primary Reference"].isin(references)]
    final = pd.concat([old_data, fact_table])
else:
    final = fact_table

In [48]:
final.to_csv(OUTPUT_PATH + "Product.csv", index=False)

c:\Users\shahmir.tariq\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\internals\blocks.py:2323: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


In [49]:
for file_name in os.listdir(REPORT_PATH):
    file_path = os.path.join(REPORT_PATH, file_name)
    if os.path.isfile(file_path) and file_name.endswith('.csv'):
        os.remove(file_path)